# Anomaly Detection

**The problem:** When we started looking into grading buildings by their GHG emissions intensity,
all of the buildings that we were going to be giving an A grade appear to be outliers, missing data
or having faulty data.

The A buildings from our [first PR](https://github.com/vkoves/electrify-chicago/pull/140/commits/14546521270ade9e47623f615af4e6868c6c9cfc) are as follows:

- https://electrifychicago.net/building/1830-n-winchester-ave/ (ID 117024)
  Multi-family housing. Reported 0 natural gas use for the past two years despite non-zero use before.

- https://electrifychicago.net/building/830-n-michigan-ave/ (ID 124236)
  Topshop and UNIQLO building, may be largely vacant, had precipitous declines

- https://electrifychicago.net/building/u-s-cellular-plaza-8430-goby-llc/ (ID 160142)
  Large decline in electricity use (3x from 2017-202), never reported gas use. Could be correct?

- https://electrifychicago.net/building/moody-bible-institute-solheim-center/ (ID 165717)
  Moody's gym, went from 2M kBTUs of natural gas to 0 in 2021 and 2022.

- https://electrifychicago.net/building/newberry-plaza-townhouse-owners-association/ (ID 172137)
  Similarly went to 0 gas from 800k KBTU,

- https://electrifychicago.net/building/u-s-cellular-plaza-8420-goby-llc/ (ID 251770)

- https://electrifychicago.net/building/4434-4444-n-damen-ave/ (ID 254001)
    Robey Condominiums, multi-family housing. Reported 0 natural gas use for the past two years
    despite non-zero use before.


## Dependencies

This notebook requires:

- pandas
- numpy
- plotly
- statsmodels
- nbformat

To install, _in this `src/data/analysis` directory_, run:

```
pip install -r requirements.txt
```

In [162]:
import pandas as pd
import numpy as np
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot
import plotly.io as pio
from plotly.subplots import make_subplots
import statsmodels.api as sm
import os
from pathlib import Path
import json

from IPython.display import Image

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

pd.set_option('display.max_columns', None)

## Set pathing

In [163]:
# get static dir for saving images
current_dir = Path.cwd()
project_root = current_dir

while True:
    if os.path.basename(project_root) == 'electrify-chicago':
        print("Success: Found 'electrify-chicago' as the base directory.")
        break
    new_root = os.path.dirname(project_root)
    if new_root == project_root:  # Reached the filesystem root
        raise FileNotFoundError("Error: 'electrify-chicago' directory not found in the path hierarchy.")
    project_root = new_root
static_blog_pth = os.path.join(project_root, 'static', 'blog', 'GHGIntensityPredictCompliance')
os.makedirs(static_blog_pth, exist_ok=True)

expected_dir_name = "analysis"
fig_dir = os.path.join(current_dir, 'output', 'compliance_analysis')

# Check if the current directory is the "analysis" folder
if current_dir.name != expected_dir_name:
    raise AssertionError(f"Expected working directory to be '{expected_dir_name}', but got '{current_dir.name}'.\n"
                         f"Please ensure you are in the correct directory.")

print(f"Current working directory is correctly set to '{current_dir}'.")

Success: Found 'electrify-chicago' as the base directory.
Current working directory is correctly set to '/home/viktor/Documents/electrify-chicago/src/data/analysis'.


### Notebook options and custom plotting function

In [164]:
reduce_memory = True # option to display plotly as static images to reduce memory, if possible
export_to_blog = False # if true, saves plots and regressions to blog static folder for website publishing

if export_to_blog:
    dirs = [static_blog_pth, fig_dir]
else:
    dirs = [fig_dir]

def show_fig(fig, reduce_memory):
    if reduce_memory:
        try:
            png_image = pio.to_image(fig, format='png')
            return (png_image, reduce_memory)

        except:
            print("Error exporting plotly to png, displaying html graph instead")
            reduce_memory = False

    if not reduce_memory:
        return (iplot(fig), reduce_memory)


## Data Prep - Loading In Reporting Buildings Over Time With Names

### Read in Data

In [165]:
# Construct the path to the CSV file (one level above the current directory)
data_path = os.path.join( current_dir.parent, 'dist', 'benchmarking-all-years.csv')

df = pd.read_csv(data_path)

# Create the "reported" column
df['Reported'] = df['GHGIntensity'].notna().astype(int)

print(f"There are {df['ID'].unique().shape[0]} unique building ids")

df['DataYear'] = df['DataYear'].astype(int)

df.head()

There are 3749 unique building ids


,ID,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported
0,252064,2020,Submitted Data,64028.0,1160.9,18.1,1.0,2.0,NaN,2384738.9,NaN,7438787.0,5594040.1,NaN,240.8,323.6,246.0,329.9,1
1,232458,2020,Submitted Data,627680.0,4871.7,7.8,1.0,1.0,22.0,16397682.8,43537490.6,NaN,NaN,NaN,95.5,146.0,100.3,150.7,1
2,254616,2020,Submitted Data,555524.0,4581.4,8.2,1.0,2.0,49.0,28606427.7,2199940.1,NaN,NaN,NaN,55.5,148.3,56.7,151.8,1
3,103812,2020,Submitted Data,130007.0,1092.1,8.4,1.0,3.0,61.0,6489281.3,1493523.2,NaN,NaN,NaN,61.4,151.8,63.0,154.8,1
4,254073,2020,Submitted Data,83000.0,295.8,3.6,1.0,4.0,100.0,1614582.3,825006.6,NaN,NaN,NaN,29.4,64.9,29.6,64.3,1


### Read in Building Benchmark Data to get Building Names

In [166]:
names_path = os.path.join( current_dir.parent, 'dist', 'building-benchmarks.csv')

building_names = pd.read_csv(names_path)[['ID', 'PropertyName' ]]
building_names.drop_duplicates(keep='first')
building_names.head()

,ID,PropertyName
0,100001,Presence SMEMC St Elizabeth Campus
1,100002,Clemente Community Academy HS -CPS
2,100019,Dixon Building
3,100068,Joffco Square
4,100148,The Jeffery Cyril Building


### Merge names to data & filter to reporting

In [195]:
df = pd.merge( df, building_names, how='left', on='ID')
df['PropertyName'] = df['PropertyName'].fillna("[Building Name Unavailable]").replace("", "[Building Name Unavailable]")

# Reorder to second column
prop_name = df.pop('PropertyName')
df.insert(1, 'PropertyName', prop_name)

df = df[df['ReportingStatus'].isin(['Submitted Data', 'Submitted'])]
df.head()

,ID,PropertyName,PropertyName_x,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported,PropertyName_y
0,252064,Mansueto Library,Mansueto Library,2020,Submitted Data,64028.0,1160.9,18.1,1.0,2.0,NaN,2384738.9,NaN,7438787.0,5594040.1,NaN,240.8,323.6,246.0,329.9,1,Mansueto Library
1,232458,Harper Square Cooperative,Harper Square Cooperative,2020,Submitted Data,627680.0,4871.7,7.8,1.0,1.0,22.0,16397682.8,43537490.6,NaN,NaN,NaN,95.5,146.0,100.3,150.7,1,Harper Square Cooperative
2,254616,Former Coyne College,Former Coyne College,2020,Submitted Data,555524.0,4581.4,8.2,1.0,2.0,49.0,28606427.7,2199940.1,NaN,NaN,NaN,55.5,148.3,56.7,151.8,1,Former Coyne College
3,103812,400 W Superior St,400 W Superior St,2020,Submitted Data,130007.0,1092.1,8.4,1.0,3.0,61.0,6489281.3,1493523.2,NaN,NaN,NaN,61.4,151.8,63.0,154.8,1,400 W Superior St
4,254073,Blue Moon Lofts,Blue Moon Lofts,2020,Submitted Data,83000.0,295.8,3.6,1.0,4.0,100.0,1614582.3,825006.6,NaN,NaN,NaN,29.4,64.9,29.6,64.3,1,Blue Moon Lofts


## Validate Data

### Test That A Building Properly Exists Across All Years

Test Newberry Plaza (ID 172137), should have 2016 - 2022 data

In [196]:
newberryData = df[df['ID'] == 172137].sort_values(by='DataYear')

assert newberryData['ID'].count() == 7, "There is not 7 years of data for Newberry Plaza, something is wrong!"

### Check that every building/year combo exists only once

In [197]:
group_counts = df.groupby(['ID', 'DataYear']).size()

print(f"There are {len(group_counts)} reports (buildings crossed with years)")

# Assert that the maximum count in any group is at most 1
assert group_counts.max() <= 1, "There are buildings with more than one row in a given year!"

There are 20609 reports (buildings crossed with years)


### Get the latest year we have data for 

In [198]:
# get buildings with zero natural gas use in past year
latestYear = df['DataYear'].max()
latestYear

2022

In [199]:
latestData = df[df['DataYear'] == latestYear]
latestData.head()

,ID,PropertyName,PropertyName_x,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported,PropertyName_y
17753,175891,[Building Name Unavailable],[Building Name Unavailable],2022,Submitted Data,172500.0,452.4,3.0,1.0,4.0,100.0,2069532.4,3384519.4,0.0,0.0,NaN,36.4,62.3,36.4,62.3,1,NaN
17755,251245,3800 N. Lake Shore Drive,3800 N. Lake Shore Drive,2022,Submitted Data,249095.0,1434.0,5.8,2.0,3.5,74.0,3345590.2,18702028.9,0.0,0.0,NaN,88.5,116.4,94.1,121.1,1,3800 N. Lake Shore Drive
21284,256658,Midpointe Apartments,Midpointe Apartments,2022,Submitted Data,393938.0,1948.4,4.9,8.0,4.0,75.0,6388293.5,20841006.3,0.0,0.0,NaN,69.1,101.0,71.9,102.6,1,Midpointe Apartments
21285,250062,RJ Quinn Academy,RJ Quinn Academy,2022,Submitted Data,66285.0,525.4,7.9,1.0,2.0,NaN,2649529.2,3322169.7,0.0,0.0,NaN,90.1,164.5,91.9,165.6,1,RJ Quinn Academy
21286,101545,[Building Name Unavailable],[Building Name Unavailable],2022,Submitted Data,51163.0,300.3,5.9,1.0,2.0,50.0,541560.1,4310877.6,0.0,0.0,NaN,94.8,118.1,101.2,123.8,1,NaN


## Approach #1 - Find Buildings That Report 0 Gas Use, But Used Gas In The Past

### Get Buildings That Didn't Use Gas In The Latest Year

In [203]:
noGasUse = latestData[latestData['NaturalGasUse'] == 0]
noGasUse.head(10)

,ID,PropertyName,PropertyName_x,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported,PropertyName_y
21287,160438,155 North Wacker,155 North Wacker,2022,Submitted Data,1484327.0,9938.8,7.1,1.0,4.0,75.0,63482520.5,0.0,0.0,29929202.7,NaN,66.4,145.8,64.4,144.9,1,155 North Wacker
21297,100179,Saint Anthony Hospital - Main Hospital,Saint Anthony Hospital - Main Hospital,2022,Submitted Data,450612.0,NaN,NaN,1.0,0.0,NaN,15191302.2,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,Saint Anthony Hospital - Main Hospital
21322,156942,Burton-Judson Courts,Burton-Judson Courts,2022,Submitted Data,201402.0,1318.2,6.5,1.0,3.0,68.0,2852269.0,0.0,14194821.6,0.0,NaN,84.6,124.4,89.7,130.6,1,Burton-Judson Courts
21354,250147,820 W Jackson Blvd,820 W Jackson Blvd,2022,Submitted Data,186957.7,1766.0,9.4,1.0,1.0,NaN,13408061.5,0.0,0.0,0.0,NaN,71.7,200.8,71.7,200.8,1,820 W Jackson Blvd
21366,101867,125 South Wacker,125 South Wacker,2022,Submitted Data,641962.0,4402.2,6.9,1.0,3.0,67.0,33422010.0,0.0,0.0,0.0,NaN,52.1,145.8,52.9,148.2,1,125 South Wacker
21368,235438,[Building Name Unavailable],[Building Name Unavailable],2022,Submitted Data,287688.0,1225.9,5.1,1.0,3.5,73.0,9307334.2,0.0,0.0,0.0,NaN,38.4,107.4,38.9,108.9,1,NaN
21373,104879,[Building Name Unavailable],[Building Name Unavailable],2022,Submitted Data,90000.0,350.0,5.4,1.0,3.5,79.0,2656906.6,0.0,0.0,0.0,NaN,40.9,114.5,40.8,114.2,1,NaN
21376,101765,77 West Wacker Drive,77 West Wacker Drive,2022,Submitted Data,1223136.0,6691.0,5.6,1.0,4.0,75.0,50799353.3,0.0,0.0,0.0,NaN,42.9,120.0,42.8,119.8,1,77 West Wacker Drive
21389,102286,Illinois Masonic Medical Office Center,Illinois Masonic Medical Office Center,2022,Submitted Data,142955.0,989.8,6.9,1.0,4.0,97.0,5781242.7,0.0,3438813.5,0.0,NaN,64.5,142.2,64.3,140.2,1,Illinois Masonic Medical Office Center
21412,101789,200 W Madison,200 W Madison,2022,Submitted Data,1026496.0,6336.6,6.3,1.0,3.0,66.0,48108398.4,0.0,0.0,0.0,NaN,48.0,134.5,48.9,136.9,1,200 W Madison


### Get Count of "Gas Free" Buildings Latest Year

In [ ]:
print('Got ' + str(noGasUse['ID'].count())
      + ' buildings that did not use gas in ' + str(latestYear))

# Check for Newberry Plaza Townhouse
buildingToValidateId = 172137;
noGasUse['ID'][noGasUse['ID'].astype(str).str.contains(str(buildingToValidateId))]

Got 1203 buildings that did not use gas in 2022


21643    172137
Name: ID, dtype: int64

### Loop Through Gas Free Buildings And See If They Used Gas in Previous Years

In [ ]:
import warnings

# Ignore Pandas warnings for this script
warnings.filterwarnings('ignore')

noGasUseIds = noGasUse['ID']
usedGasBefore = df[df['DataYear'] < latestYear][df['NaturalGasUse'] > 0][df['ID'].isin(noGasUseIds)]

gasAnomalyIds = usedGasBefore['ID'].unique()

print('Got ' + str(len(gasAnomalyIds))
      + ' gas-anomaly buildings (used gas before, but not in latest year)! \n')

# Check if our building is in the dataset
print('Newberry Plaza (ID ' + str(buildingToValidateId) + ') in data? '
      + str(buildingToValidateId in gasAnomalyIds))

print('\nGas Anomaly IDs:')

gasAnomalyIds

Got 726 gas-anomaly buildings (used gas before, but not in latest year)! 

Newberry Plaza (ID 172137) in data? True

Gas Anomaly IDs:


array([129621, 250020, 254001, 254102, 147565, 165664, 173857, 160208,
       121374, 160186, 254164, 173888, 251429, 172485, 251588, 175960,
       251491, 251683, 160347, 251401, 165819, 159892, 260149, 251470,
       251536, 250002, 251461, 251389, 254996, 175817, 255020, 132904,
       251447, 251607, 255013, 172611, 140247, 242935, 175157, 173632,
       256639, 251476, 104789, 100002, 250043, 250015, 100470, 174228,
       256429, 251628, 251760, 251409, 175785, 251406, 161872, 175914,
       251581, 250022, 172703, 116750, 101971, 135050, 254068, 255028,
       102587, 175954, 255004, 101872, 103602, 101980, 102987, 113670,
       105600, 252028, 251384, 251794, 256802, 256557, 254169, 101657,
       254171, 119987, 120299, 172256, 250047, 115942, 251695, 175754,
       251550, 256420, 251694, 250016, 254162, 173811, 251564, 160429,
       120010, 253986, 134783, 252068, 100395, 164512, 125148, 254378,
       251580, 251464, 161305, 251661, 132870, 256423, 156500, 175995,
      

### Did This Catch Our Bad "A" Buildings?

In [ ]:
###
### Should catch 117024 ✅, 165717 ✅, 172137✅, 254001 ✅
AGradedBuildingIds = [117024, 124236, 160142, 165717, 172137, 251770, 254001]
AGradedBuildingIds = set(AGradedBuildingIds)
AGradedBuildingIds.intersection(gasAnomalyIds)

{117024, 165717, 172137, 254001}

## Approach #2 - Find Anomalies By Percentage Change In Gas Use 

Some buildings (like [Keating Hall](https://electrifychicago.net/building/keating-hall/)) didn't have their gas use drop
all the way to zero, but had _huge_ drops in gas use (e.g 456 million kBTUs in 2020 )

In [ ]:
gas_tracked = df.copy()

# Move gas use to third column (after year)
nat_gas_use = gas_tracked.pop('NaturalGasUse')
gas_tracked.insert(3, 'NaturalGasUse', nat_gas_use)

# 1. Sort by ID and DataYear (Important!)
gas_tracked.sort_values(['ID', 'DataYear'], inplace=True)

# 2. Calculate the difference in years between consecutive rows within each ID group
gas_tracked['Year_Difference'] = gas_tracked.groupby('ID')['DataYear'].diff()

# 3. Calculate the percentage change, ignoring gaps (where the time difference between the two rows
# would be > 1 year)
def calculate_change(group):
    group['NatGasPrcntChange'] = np.where(
        group['Year_Difference'] == 1,  # Check if the year difference is 1
        group['NaturalGasUse'].pct_change() * 100,
        np.nan  # If the year difference is not 1, set to NaN (or 0, or another value)
    )
    return group

gas_tracked = gas_tracked.groupby('ID').apply(calculate_change)

# 4. Handle NaNs (from the initial pct_change and from year gaps)
gas_tracked['NatGasPrcntChange'].fillna(0, inplace=True) # or another value

# Insert our new column at column 4 for readability
gas_prcnt_change = gas_tracked.pop('NatGasPrcntChange')
gas_tracked.insert(4, 'NatGasPrcntChange', gas_prcnt_change)


# # Get Merch Mart (ID 103656), which has 10 years of clean data
notable_building_ids = { 'MerchMart': 103656, 'Keating': 256434 }

# sample = gas_tracked[gas_tracked['ID'] == notable_building_ids['MerchMart']]
sample = gas_tracked.sort_values( by='NatGasPrcntChange', ascending=True)

sample.head(10)

ID                                      PropertyName  \
ID                                                                       
254051 19821  254051                       [Building Name Unavailable]   
       23731  254051                       [Building Name Unavailable]   
105820 21265  105820                        Quality Inn Midway Airport   
251613 17928  251613  SoutHigh School ide Occupational Academy HS -CPS   
165664 23719  165664         DANK Haus German American Cultural Center   
244817 20321  244817                                        The Towers   
173812 18784  173812                       [Building Name Unavailable]   
105034 21439  105034                         Carmen Marine Cooperative   
100372 18212  100372                                Holy Cross Hospial   
250117 20738  250117                                          S1-Sinai   

              DataYear  NaturalGasUse  NatGasPrcntChange ReportingStatus  \
ID                                                                         
254051 19821      2021     -4774717.8        -316.130939       Submitted   
       23731      2022      6313744.3        -232.232826  Submitted Data   
105820 21265      2021            0.0        -100.000000       Submitted   
251613 17928      2021            0.0        -100.000000       Submitted   
165664 23719      2022            0.0        -100.000000  Submitted Data   
244817 20321      2021            0.0        -100.000000       Submitted   
173812 18784      2021            0.0        -100.000000       Submitted   
105034 21439      2022            0.0        -100.000000  Submitted Data   
100372 18212      2021            0.0        -100.000000       Submitted   
250117 20738      2021            0.0        -100.000000       Submitted   

              GrossFloorArea  TotalGHGEmissions  GHGIntensity  \
ID                                                              
254051 19821         83678.0             1201.5          14.4   
       23731        257906.0             1291.5           5.0   
105820 21265         86770.0                NaN           NaN   
251613 17928         54740.0                NaN           NaN   
165664 23719         61042.0                NaN           NaN   
244817 20321        217800.0                NaN           NaN   
173812 18784        117514.0                NaN           NaN   
105034 21439        303602.0                0.0           0.0   
100372 18212        442222.0                NaN           NaN   
250117 20738       1157944.0                NaN           NaN   

              NumberOfBuildings  ChicagoEnergyRating  ENERGYSTARScore  \
ID                                                                      
254051 19821                1.0                  4.0             94.0   
       23731                1.0                  NaN              NaN   
105820 21265                NaN                  0.0              NaN   
251613 17928                1.0                  0.0              NaN   
165664 23719                1.0                  NaN              NaN   
244817 20321                NaN                  0.0              NaN   
173812 18784                NaN                  0.0              NaN   
105034 21439                1.0                  0.0              NaN   
100372 18212                1.0                  0.0              NaN   
250117 20738                1.0                  0.0              NaN   

              ElectricityUse  DistrictSteamUse  DistrictChilledWaterUse  \
ID                                                                        
254051 19821      10187894.2               0.0                      0.0   
       23731       7258900.0               0.0                      0.0   
105820 21265             0.0               0.0                      0.0   
251613 17928             0.0               0.0                      0.0   
165664 23719        549191.2               0.0                      0.0   
244817 20321             0.0     

## Approach #3 - Find Buildings With GHG Intensity But No Electricity Use

In [ ]:
no_electricity_use_buildings = latestData[latestData['GHGIntensity'].notna()][latestData['ElectricityUse'].isin([0, np.nan])]
no_electricity_use_buildings.head()

,ID,PropertyName,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported
21439,105034,Carmen Marine Cooperative,2022,Submitted Data,303602.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
22870,104431,[Building Name Unavailable],2022,Submitted Data,16333.0,4.8,0.3,1.0,4.0,100.0,0.0,91108.8,0.0,0.0,NaN,5.6,5.9,5.6,5.9,1


Since there are only two buildings matching this, we can safely ignore that for anomaly detection.